# 📊 Статистические тесты: от теории к практике

**Цель занятия:** Научиться проверять гипотезы на данных с помощью статистических тестов

**План:**
1. 🍕 Разминка на данных доставки пиццы
2. 🏪 Практика на реальных данных X5 Retail
3. 🎯 Самостоятельная работа

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import mannwhitneyu, pearsonr, spearmanr, ttest_ind, ttest_rel
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
%matplotlib inline

print("🎯 Добро пожаловать в мир статистических тестов!")

---
## 🍕 Часть 1: Примеры на данных доставки пиццы

Представим, что мы анализируем службу доставки пиццы и хотим понять:
- Влияет ли день недели на время доставки?
- Есть ли связь между расстоянием и временем доставки?
- Отличаются ли чаевые в будни и выходные?

### 1.1 Генерация данных

In [ ]:
# Создаем датасет доставки пиццы
np.random.seed(42)

n_orders = 500

# Генерируем данные
days = np.random.choice(['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс'], n_orders)
is_weekend = np.isin(days, ['Сб', 'Вс'])

# Время доставки зависит от дня недели (в выходные дольше)
delivery_time = np.where(is_weekend, 
                         np.random.normal(35, 9, n_orders),  # выходные
                         np.random.normal(28, 7, n_orders))  # будни

# Расстояние
distance = np.random.exponential(3, n_orders)

# Время доставки коррелирует с расстоянием
delivery_time = delivery_time + distance * 2 + np.random.normal(0, 2, n_orders)

# Чаевые выше в выходные
tips = np.where(is_weekend,
                np.random.normal(150, 50, n_orders),  # выходные
                np.random.normal(100, 40, n_orders))  # будни

pizza_data = pd.DataFrame({
    'order_id': range(1, n_orders + 1),
    'day_of_week': days,
    'is_weekend': is_weekend,
    'delivery_time': delivery_time,  # минуты
    'distance': distance,  # км
    'tips': np.maximum(0, tips),  # рубли, не меньше 0
    'pizza_size': np.random.choice(['Маленькая', 'Средняя', 'Большая'], n_orders, p=[0.2, 0.5, 0.3]),
    'rating': np.random.choice([3, 4, 5], n_orders, p=[0.1, 0.3, 0.6])
})

print(f"📦 Создан датасет из {len(pizza_data)} заказов")
pizza_data.head(10)

In [ ]:
# Быстрый обзор данных
pizza_data.describe()

---
## 📈 1.2 T-test (Т-тест Стьюдента)

**Вопрос:** Отличается ли среднее время доставки в будни и выходные?

**Когда использовать:**
- Сравниваем средние двух групп
- Данные должны быть примерно нормально распределены
- Независимые наблюдения

**Гипотезы:**
- H₀ (нулевая): средние равны (нет разницы)
- H₁ (альтернативная): средние различаются

In [ ]:
# Разделим данные на две группы
weekday_delivery = pizza_data[~pizza_data['is_weekend']]['delivery_time']
weekend_delivery = pizza_data[pizza_data['is_weekend']]['delivery_time']

print(f"Будни: n={len(weekday_delivery)}, среднее={weekday_delivery.mean():.2f} мин")
print(f"Выходные: n={len(weekend_delivery)}, среднее={weekend_delivery.mean():.2f} мин")
print(f"\nРазница: {weekend_delivery.mean() - weekday_delivery.mean():.2f} мин")

In [ ]:
# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Гистограммы
axes[0].hist(weekday_delivery, bins=30, alpha=0.7, label='Будни', color='skyblue')
axes[0].hist(weekend_delivery, bins=30, alpha=0.7, label='Выходные', color='salmon')
axes[0].axvline(weekday_delivery.mean(), color='blue', linestyle='--', linewidth=2, label='Среднее (будни)')
axes[0].axvline(weekend_delivery.mean(), color='red', linestyle='--', linewidth=2, label='Среднее (выходные)')
axes[0].set_xlabel('Время доставки (мин)')
axes[0].set_ylabel('Частота')
axes[0].set_title('Распределение времени доставки')
axes[0].legend()

# Boxplot
data_to_plot = [weekday_delivery, weekend_delivery]
axes[1].boxplot(data_to_plot, labels=['Будни', 'Выходные'])
axes[1].set_ylabel('Время доставки (мин)')
axes[1].set_title('Сравнение времени доставки')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Проводим t-test
t_statistic, p_value = ttest_ind(weekday_delivery, weekend_delivery)

print("="*60)
print("📊 РЕЗУЛЬТАТЫ T-ТЕСТА")
print("="*60)
print(f"t-статистика: {t_statistic:.4f}")
print(f"p-value: {p_value:.4f}")
print("="*60)

alpha = 0.05
if p_value < alpha:
    print(f"✅ ВЫВОД: p-value ({p_value:.4f}) < {alpha}")
    print("   Отвергаем H₀: средние РАЗЛИЧАЮТСЯ")
    print(f"   Время доставки в выходные действительно отличается!")
else:
    print(f"❌ ВЫВОД: p-value ({p_value:.4f}) >= {alpha}")
    print("   Не можем отвергнуть H₀: нет достаточных оснований")
    print("   утверждать, что средние различаются")

### 💡 Интерпретация p-value

**p-value** — это вероятность получить такие же (или более экстремальные) результаты, если H₀ верна.

- **p < 0.05** → результат статистически значим (отвергаем H₀)
- **p ≥ 0.05** → результат не значим (не отвергаем H₀)

⚠️ **Важно:** p-value НЕ говорит о размере эффекта, только о его статистической значимости!

---
## 📊 1.3 Тест Манна-Уитни (Mann-Whitney U-test)

**Вопрос:** Отличаются ли чаевые в будни и выходные?

**Когда использовать:**
- Непараметрическая альтернатива t-тесту
- Когда данные НЕ нормально распределены
- Или есть выбросы
- Сравниваем медианы/распределения двух групп

In [ ]:
# Посмотрим на распределение чаевых
weekday_tips = pizza_data[~pizza_data['is_weekend']]['tips']
weekend_tips = pizza_data[pizza_data['is_weekend']]['tips']

print("Статистики по чаевым:")
print(f"Будни: медиана={weekday_tips.median():.2f}₽, среднее={weekday_tips.mean():.2f}₽")
print(f"Выходные: медиана={weekend_tips.median():.2f}₽, среднее={weekend_tips.mean():.2f}₽")

In [ ]:
# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Гистограммы
axes[0].hist(weekday_tips, bins=30, alpha=0.7, label='Будни', color='lightgreen')
axes[0].hist(weekend_tips, bins=30, alpha=0.7, label='Выходные', color='orange')
axes[0].set_xlabel('Чаевые (₽)')
axes[0].set_ylabel('Частота')
axes[0].set_title('Распределение чаевых')
axes[0].legend()

# Boxplot
pizza_data.boxplot(column='tips', by='is_weekend', ax=axes[1])
axes[1].set_xticklabels(['Будни', 'Выходные'])
axes[1].set_ylabel('Чаевые (₽)')
axes[1].set_title('Сравнение чаевых')
axes[1].set_xlabel('')
plt.suptitle('')  # убираем автоматический заголовок

plt.tight_layout()
plt.show()

In [ ]:
# Тест Манна-Уитни
u_statistic, p_value_mw = mannwhitneyu(weekday_tips, weekend_tips, alternative='two-sided')

print("="*60)
print("📊 РЕЗУЛЬТАТЫ ТЕСТА МАННА-УИТНИ")
print("="*60)
print(f"U-статистика: {u_statistic:.4f}")
print(f"p-value: {p_value_mw:.4f}")
print("="*60)

if p_value_mw < 0.05:
    print(f"✅ ВЫВОД: p-value ({p_value_mw:.4f}) < 0.05")
    print("   Распределения чаевых РАЗЛИЧАЮТСЯ между буднями и выходными")
else:
    print(f"❌ ВЫВОД: p-value ({p_value_mw:.4f}) >= 0.05")
    print("   Нет достаточных оснований утверждать, что распределения различаются")

### 🤔 T-test vs Mann-Whitney: когда что использовать?

| Критерий | T-test | Mann-Whitney |
|----------|--------|-------------|
| Предположения | Нормальность | Нет предположений |
| Сравнивает | Средние | Медианы/ранги |
| Чувствительность к выбросам | Высокая | Низкая |
| Мощность при нормальности | Выше | Ниже |
| Когда использовать | Нормальные данные | Ненормальные данные, выбросы |

---
## 📈 1.4 Корреляция (Correlation)

**Вопрос:** Есть ли связь между расстоянием доставки и временем?

**Типы корреляции:**
- **Пирсон** — линейная связь (для нормальных данных)
- **Спирман** — монотонная связь (непараметрический)

In [ ]:
# Визуализация связи
plt.figure(figsize=(10, 6))
plt.scatter(pizza_data['distance'], pizza_data['delivery_time'], alpha=0.5, s=50)
plt.xlabel('Расстояние (км)', fontsize=12)
plt.ylabel('Время доставки (мин)', fontsize=12)
plt.title('Зависимость времени доставки от расстояния', fontsize=14)
plt.grid(True, alpha=0.3)

# Добавим линию тренда
z = np.polyfit(pizza_data['distance'], pizza_data['delivery_time'], 1)
p = np.poly1d(z)
plt.plot(pizza_data['distance'].sort_values(), 
         p(pizza_data['distance'].sort_values()), 
         "r--", linewidth=2, label='Линия тренда')
plt.legend()
plt.show()

In [ ]:
# Корреляция Пирсона
pearson_corr, pearson_p = pearsonr(pizza_data['distance'], pizza_data['delivery_time'])

print("="*60)
print("📊 КОРРЕЛЯЦИЯ ПИРСОНА")
print("="*60)
print(f"Коэффициент корреляции: {pearson_corr:.4f}")
print(f"p-value: {pearson_p:.6f}")
print("="*60)

# Интерпретация силы связи
abs_corr = abs(pearson_corr)
if abs_corr < 0.3:
    strength = "слабая"
elif abs_corr < 0.7:
    strength = "умеренная"
else:
    strength = "сильная"

direction = "положительная" if pearson_corr > 0 else "отрицательная"

print(f"\n✅ Обнаружена {strength} {direction} связь")
if pearson_p < 0.05:
    print(f"✅ Связь статистически значима (p < 0.05)")
else:
    print(f"❌ Связь статистически НЕ значима (p >= 0.05)")

In [ ]:
# Корреляция Спирмана (для сравнения)
spearman_corr, spearman_p = spearmanr(pizza_data['distance'], pizza_data['delivery_time'])

print("="*60)
print("📊 КОРРЕЛЯЦИЯ СПИРМАНА")
print("="*60)
print(f"Коэффициент корреляции: {spearman_corr:.4f}")
print(f"p-value: {spearman_p:.6f}")
print("="*60)

print(f"\n💡 Сравнение: Пирсон={pearson_corr:.3f} vs Спирман={spearman_corr:.3f}")
print("   Если значения близки → связь линейная")
print("   Если Спирман >> Пирсон → связь нелинейная")

### 💡 Интерпретация коэффициента корреляции

| Значение r | Интерпретация |
|------------|---------------|
| 0.0 - 0.3 | Слабая связь |
| 0.3 - 0.7 | Умеренная связь |
| 0.7 - 1.0 | Сильная связь |

⚠️ **Важно:** Корреляция ≠ Причинность!

In [ ]:
# Корреляционная матрица для всех числовых переменных
numeric_cols = ['delivery_time', 'distance', 'tips', 'rating']
corr_matrix = pizza_data[numeric_cols].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Корреляционная матрица', fontsize=14)
plt.tight_layout()
plt.show()

---
## 🎲 1.5 Bootstrap (Бутстрап)

**Вопрос:** Насколько точна наша оценка среднего времени доставки?

**Идея бутстрапа:**
1. Создаем много (например, 10000) выборок с возвращением из исходных данных
2. Для каждой выборки вычисляем статистику (среднее, медиану и т.д.)
3. Получаем распределение статистики и доверительный интервал

**Преимущества:**
- Не требует предположений о распределении
- Работает для любых статистик
- Даёт оценку неопределённости

In [ ]:
def bootstrap_statistic(data, statistic_func, n_bootstrap=10000, confidence_level=0.95):
    """
    Вычисляет статистику и её доверительный интервал методом бутстрапа
    
    Parameters:
    -----------
    data : array-like
        Исходные данные
    statistic_func : callable
        Функция для вычисления статистики (например, np.mean)
    n_bootstrap : int
        Количество бутстрап-выборок
    confidence_level : float
        Уровень доверия (например, 0.95 для 95% ДИ)
    
    Returns:
    --------
    dict с результатами
    """
    bootstrap_stats = []
    n = len(data)
    
    for _ in range(n_bootstrap):
        # Создаем выборку с возвращением
        bootstrap_sample = np.random.choice(data, size=n, replace=True)
        # Вычисляем статистику
        stat = statistic_func(bootstrap_sample)
        bootstrap_stats.append(stat)
    
    bootstrap_stats = np.array(bootstrap_stats)
    
    # Вычисляем доверительный интервал
    alpha = 1 - confidence_level
    lower = np.percentile(bootstrap_stats, alpha/2 * 100)
    upper = np.percentile(bootstrap_stats, (1 - alpha/2) * 100)
    
    return {
        'original_statistic': statistic_func(data),
        'bootstrap_mean': np.mean(bootstrap_stats),
        'bootstrap_std': np.std(bootstrap_stats),
        'confidence_interval': (lower, upper),
        'bootstrap_distribution': bootstrap_stats
    }

print("✅ Функция бутстрапа готова!")

In [ ]:
# Применяем бутстрап для среднего времени доставки
delivery_times = pizza_data['delivery_time'].values

result = bootstrap_statistic(delivery_times, np.mean, n_bootstrap=10000)

print("="*60)
print("📊 РЕЗУЛЬТАТЫ БУТСТРАПА")
print("="*60)
print(f"Исходная оценка среднего: {result['original_statistic']:.2f} мин")
print(f"Бутстрап-оценка среднего: {result['bootstrap_mean']:.2f} мин")
print(f"Стандартная ошибка: {result['bootstrap_std']:.2f} мин")
print(f"95% доверительный интервал: [{result['confidence_interval'][0]:.2f}, {result['confidence_interval'][1]:.2f}] мин")
print("="*60)
print(f"\n💡 Интерпретация: с вероятностью 95% истинное среднее время доставки")
print(f"   находится в интервале от {result['confidence_interval'][0]:.2f} до {result['confidence_interval'][1]:.2f} минут")

In [ ]:
# Визуализация бутстрап-распределения
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(result['bootstrap_distribution'], bins=50, edgecolor='black', alpha=0.7)
plt.axvline(result['original_statistic'], color='red', linestyle='--', linewidth=2, label='Исходная оценка')
plt.axvline(result['confidence_interval'][0], color='green', linestyle='--', linewidth=2, label='95% ДИ')
plt.axvline(result['confidence_interval'][1], color='green', linestyle='--', linewidth=2)
plt.xlabel('Среднее время доставки (мин)')
plt.ylabel('Частота')
plt.title('Бутстрап-распределение среднего')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(delivery_times, bins=30, edgecolor='black', alpha=0.7, color='skyblue')
plt.axvline(result['original_statistic'], color='red', linestyle='--', linewidth=2, label='Среднее')
plt.xlabel('Время доставки (мин)')
plt.ylabel('Частота')
plt.title('Исходное распределение данных')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Бутстрап для разницы средних (будни vs выходные)
def diff_means(data1, data2, n_bootstrap=10000):
    """Вычисляет доверительный интервал для разницы средних"""
    diffs = []
    
    for _ in range(n_bootstrap):
        sample1 = np.random.choice(data1, size=len(data1), replace=True)
        sample2 = np.random.choice(data2, size=len(data2), replace=True)
        diffs.append(np.mean(sample2) - np.mean(sample1))
    
    diffs = np.array(diffs)
    lower = np.percentile(diffs, 2.5)
    upper = np.percentile(diffs, 97.5)
    
    return {
        'observed_diff': np.mean(data2) - np.mean(data1),
        'ci_lower': lower,
        'ci_upper': upper,
        'distribution': diffs
    }

weekday_delivery = pizza_data[~pizza_data['is_weekend']]['delivery_time'].values
weekend_delivery = pizza_data[pizza_data['is_weekend']]['delivery_time'].values

diff_result = diff_means(weekday_delivery, weekend_delivery)

print("="*60)
print("📊 БУТСТРАП: РАЗНИЦА ВРЕМЕНИ ДОСТАВКИ")
print("="*60)
print(f"Наблюдаемая разница: {diff_result['observed_diff']:.2f} мин")
print(f"95% ДИ: [{diff_result['ci_lower']:.2f}, {diff_result['ci_upper']:.2f}] мин")
print("="*60)

if diff_result['ci_lower'] > 0 or diff_result['ci_upper'] < 0:
    print(f"\n✅ Доверительный интервал НЕ включает 0")
    print(f"   Разница статистически значима!")
    print(f"   В выходные доставка занимает на {diff_result['observed_diff']:.2f} минут больше")
else:
    print(f"\n❌ Доверительный интервал включает 0")
    print(f"   Не можем утверждать, что есть значимая разница")

In [ ]:
# Визуализация
plt.figure(figsize=(10, 6))
plt.hist(diff_result['distribution'], bins=50, edgecolor='black', alpha=0.7, color='coral')
plt.axvline(diff_result['observed_diff'], color='red', linestyle='--', linewidth=2, label='Наблюдаемая разница')
plt.axvline(diff_result['ci_lower'], color='green', linestyle='--', linewidth=2, label='95% ДИ')
plt.axvline(diff_result['ci_upper'], color='green', linestyle='--', linewidth=2)
plt.axvline(0, color='black', linestyle='-', linewidth=1, alpha=0.5, label='Нет разницы')
plt.xlabel('Разница во времени доставки (выходные - будни), мин')
plt.ylabel('Частота')
plt.title('Бутстрап-распределение разницы средних')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 🏪 Часть 2: Практика на данных X5 Retail

Теперь применим изученные методы к реальным данным магазинов «Пятёрочка»

### 2.1 Загрузка и изучение данных

In [ ]:
# Загрузка данных
df = pd.read_excel("https://dano.hse.ru/mirror/pubs/share/1102761259.xlsx")

print(f"📦 Загружено {len(df):,} записей")
print(f"📊 Количество уникальных магазинов: {df['new_id'].nunique()}")
print(f"📅 Месяцев наблюдений: {df['Месяц'].nunique()}")

df.head()

In [ ]:
# Информация о датасете
df.info()

In [ ]:
# Основные статистики
df.describe()

In [ ]:
# Проверим категориальные переменные
print("Категориальные переменные:\n")
categorical_cols = df.select_dtypes(include='object').columns

for col in categorical_cols:
    print(f"\n{col}:")
    print(df[col].value_counts())

### 2.2 Пример анализа: Влияет ли размер магазина на трафик?

In [ ]:
# Визуализация трафика по размеру магазина
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
df.boxplot(column='Трафик', by='Торговая площадь, категориальный', ax=plt.gca())
plt.ylabel('Трафик (чел/мес)')
plt.xlabel('Размер магазина')
plt.title('Распределение трафика по размеру магазина')
plt.suptitle('')

plt.subplot(1, 2, 2)
df.boxplot(column='Средний чек', by='Торговая площадь, категориальный', ax=plt.gca())
plt.ylabel('Средний чек (₽)')
plt.xlabel('Размер магазина')
plt.title('Распределение среднего чека по размеру магазина')
plt.suptitle('')

plt.tight_layout()
plt.show()

In [ ]:
# Сравним большие и маленькие магазины
small_stores = df[df['Торговая площадь, категориальный'] == 'маленький']['Трафик']
big_stores = df[df['Торговая площадь, категориальный'] == 'большой']['Трафик']

print("Статистики по трафику:")
print(f"Маленькие магазины: среднее={small_stores.mean():.0f}, медиана={small_stores.median():.0f}")
print(f"Большие магазины: среднее={big_stores.mean():.0f}, медиана={big_stores.median():.0f}")
print(f"\nРазница в среднем: {big_stores.mean() - small_stores.mean():.0f} посетителей")

In [ ]:
# T-test
t_stat, p_val = ttest_ind(small_stores, big_stores)

print("="*60)
print("📊 T-TEST: ТРАФИК В МАЛЕНЬКИХ VS БОЛЬШИХ МАГАЗИНАХ")
print("="*60)
print(f"t-статистика: {t_stat:.4f}")
print(f"p-value: {p_val:.10f}")
print("="*60)

if p_val < 0.05:
    print(f"\n✅ Трафик статистически значимо различается!")
    if big_stores.mean() > small_stores.mean():
        print(f"   Большие магазины имеют выше трафик")
else:
    print(f"\n❌ Нет статистически значимой разницы в трафике")

In [ ]:
# Mann-Whitney (альтернативный тест)
u_stat, p_val_mw = mannwhitneyu(small_stores, big_stores)

print("="*60)
print("📊 MANN-WHITNEY: ТРАФИК В МАЛЕНЬКИХ VS БОЛЬШИХ МАГАЗИНАХ")
print("="*60)
print(f"U-статистика: {u_stat:.4f}")
print(f"p-value: {p_val_mw:.10f}")
print("="*60)

### 2.3 Пример: Корреляция между характеристиками магазина

In [ ]:
# Выберем интересующие числовые переменные
correlation_vars = [
    'Трафик',
    'Средний чек',
    'Численность населения',
    'Количество домохозяйств',
    'Трафик пеший, в час',
    'Трафик авто, в час',
    'Остановки (300 м)',
    'Продуктовые магазины (500 м)',
    'Пятерочки (500 м)'
]

# Корреляционная матрица
corr_matrix = df[correlation_vars].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Корреляционная матрица характеристик магазинов', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Детальный анализ: связь трафика с пешеходным потоком
corr_pearson, p_pearson = pearsonr(df['Трафик пеший, в час'], df['Трафик'])
corr_spearman, p_spearman = spearmanr(df['Трафик пеший, в час'], df['Трафик'])

print("="*60)
print("📊 КОРРЕЛЯЦИЯ: ПЕШЕХОДНЫЙ ПОТОК И ТРАФИК МАГАЗИНА")
print("="*60)
print(f"Пирсон: r={corr_pearson:.4f}, p-value={p_pearson:.6f}")
print(f"Спирман: ρ={corr_spearman:.4f}, p-value={p_spearman:.6f}")
print("="*60)

if p_pearson < 0.05:
    print(f"\n✅ Обнаружена статистически значимая корреляция!")
    if abs(corr_pearson) > 0.7:
        print(f"   Сильная связь между пешеходным потоком и трафиком магазина")
    elif abs(corr_pearson) > 0.3:
        print(f"   Умеренная связь между пешеходным потоком и трафиком магазина")
    else:
        print(f"   Слабая связь между пешеходным потоком и трафиком магазина")

In [ ]:
# Визуализация
plt.figure(figsize=(10, 6))
plt.scatter(df['Трафик пеший, в час'], df['Трафик'], alpha=0.3, s=10)
plt.xlabel('Пешеходный трафик (чел/час)', fontsize=12)
plt.ylabel('Трафик магазина (чел/мес)', fontsize=12)
plt.title(f'Связь пешеходного потока и трафика магазина\n(r={corr_pearson:.3f}, p<0.001)', fontsize=14)
plt.grid(True, alpha=0.3)

# Добавим линию тренда
z = np.polyfit(df['Трафик пеший, в час'], df['Трафик'], 1)
p = np.poly1d(z)
x_line = np.linspace(df['Трафик пеший, в час'].min(), df['Трафик пеший, в час'].max(), 100)
plt.plot(x_line, p(x_line), "r--", linewidth=2, label='Линия тренда')
plt.legend()
plt.tight_layout()
plt.show()

### 2.4 Пример: Бутстрап для оценки среднего чека

In [ ]:
# Возьмем случайную выборку для ускорения расчетов
sample_checks = df['Средний чек'].sample(n=5000, random_state=42).values

# Бутстрап
bootstrap_result = bootstrap_statistic(sample_checks, np.mean, n_bootstrap=5000)

print("="*60)
print("📊 БУТСТРАП: СРЕДНИЙ ЧЕК")
print("="*60)
print(f"Оценка среднего чека: {bootstrap_result['original_statistic']:.2f}₽")
print(f"Стандартная ошибка: {bootstrap_result['bootstrap_std']:.2f}₽")
print(f"95% доверительный интервал: [{bootstrap_result['confidence_interval'][0]:.2f}, {bootstrap_result['confidence_interval'][1]:.2f}]₽")
print("="*60)
print(f"\n💡 С вероятностью 95% истинный средний чек находится")
print(f"   в диапазоне {bootstrap_result['confidence_interval'][0]:.2f}-{bootstrap_result['confidence_interval'][1]:.2f} рублей")

In [ ]:
# Визуализация
plt.figure(figsize=(10, 6))
plt.hist(bootstrap_result['bootstrap_distribution'], bins=50, edgecolor='black', alpha=0.7, color='lightcoral')
plt.axvline(bootstrap_result['original_statistic'], color='red', linestyle='--', linewidth=2, label='Оценка среднего')
plt.axvline(bootstrap_result['confidence_interval'][0], color='green', linestyle='--', linewidth=2, label='95% ДИ')
plt.axvline(bootstrap_result['confidence_interval'][1], color='green', linestyle='--', linewidth=2)
plt.xlabel('Средний чек (₽)', fontsize=12)
plt.ylabel('Частота', fontsize=12)
plt.title('Бутстрап-распределение среднего чека', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 🎯 Часть 3: САМОСТОЯТЕЛЬНАЯ РАБОТА

Теперь ваша очередь! Проверьте следующие гипотезы на данных X5:

### Задание 1: Влияет ли дата открытия магазина на средний чек?

**Что нужно сделать:**
1. Разделите данные на группы по переменной `Дата открытия, категориальный`
2. Проведите t-test или Mann-Whitney для сравнения среднего чека
3. Визуализируйте результаты (boxplot или гистограммы)
4. Сделайте вывод

**Подсказка:** используйте код из примеров выше

In [ ]:
# ВАШ КОД ЗДЕСЬ

# 1. Посмотрите на распределение по дате открытия
print(df['Дата открытия, категориальный'].value_counts())

# 2. Разделите данные
# new_stores = ...
# old_stores = ...

# 3. Проведите тест

# 4. Визуализируйте

# 5. Сделайте выводы

### Задание 2: Есть ли связь между количеством остановок и трафиком?

**Что нужно сделать:**
1. Вычислите корреляцию Пирсона и Спирмана между `Остановки (300 м)` и `Трафик`
2. Проверьте статистическую значимость
3. Постройте scatter plot
4. Интерпретируйте результаты

In [ ]:
# ВАШ КОД ЗДЕСЬ

# 1. Корреляция
# pearson_corr, pearson_p = ...
# spearman_corr, spearman_p = ...

# 2. Визуализация

# 3. Выводы

### Задание 3: Отличается ли трафик в магазинах в разных регионах?

**Что нужно сделать:**
1. Выберите 2-3 региона с наибольшим количеством магазинов
2. Сравните трафик между регионами (попарно или с помощью ANOVA)
3. Визуализируйте результаты
4. Используйте бутстрап для оценки доверительных интервалов средних по регионам

In [ ]:
# ВАШ КОД ЗДЕСЬ

# 1. Посмотрите на распределение магазинов по регионам
print(df['Регион'].value_counts().head(10))

# 2. Выберите регионы и сравните

# 3. Визуализация

# 4. Бутстрап

### Задание 4: Влияет ли конкуренция на средний чек?

**Что нужно сделать:**
1. Используйте переменную `Продуктовые магазины (500 м)` как меру конкуренции
2. Разделите магазины на группы: низкая/средняя/высокая конкуренция
3. Сравните средний чек между группами
4. Проверьте корреляцию между количеством конкурентов и средним чеком

In [ ]:
# ВАШ КОД ЗДЕСЬ

# 1. Посмотрите на распределение количества конкурентов
df['Продуктовые магазины (500 м)'].describe()

# 2. Создайте категории конкуренции
# df['Конкуренция'] = pd.cut(df['Продуктовые магазины (500 м)'], 
#                             bins=[0, 5, 10, 100], 
#                             labels=['Низкая', 'Средняя', 'Высокая'])

# 3. Анализ

### Задание 5*: Свободное исследование (бонус)

**Придумайте свою гипотезу** на основе данных и проверьте её!

Примеры идей:
- Влияет ли наличие школ рядом на трафик?
- Есть ли связь между автомобильным потоком и средним чеком?
- Отличается ли трафик в зависимости от численности населения?
- Влияет ли каннибализация (наличие других Пятёрочек рядом) на трафик?

**Требования:**
1. Сформулируйте гипотезу
2. Выберите подходящий статистический тест
3. Проведите анализ
4. Визуализируйте результаты
5. Сделайте выводы

In [ ]:
# ВАШ КОД ЗДЕСЬ

# Ваша гипотеза:
# H0: ...
# H1: ...

# Анализ:

---
## 📚 Шпаргалка: Какой тест использовать?

| Ситуация | Тест | Условия применения |
|----------|------|-------------------|
| Сравнение средних 2 групп (нормальные данные) | **t-test** | Данные нормальны, независимы |
| Сравнение средних 2 групп (ненормальные данные) | **Mann-Whitney** | Нет требований к нормальности |
| Сравнение средних 3+ групп (нормальные) | **ANOVA** | Данные нормальны, равные дисперсии |
| Сравнение средних 3+ групп (ненормальные) | **Kruskal-Wallis** | Нет требований к нормальности |
| Связь между двумя числовыми переменными | **Корреляция Пирсона** | Линейная связь, нормальность |
| Связь между двумя числовыми переменными (ненормальные) | **Корреляция Спирмана** | Монотонная связь |
| Оценка неопределённости любой статистики | **Bootstrap** | Универсальный метод |

### 🎯 Алгоритм выбора теста:

1. **Что сравниваем?**
   - Средние/медианы → t-test / Mann-Whitney
   - Связь переменных → Корреляция
   - Доверительный интервал → Bootstrap

2. **Сколько групп?**
   - 2 группы → t-test / Mann-Whitney
   - 3+ группы → ANOVA / Kruskal-Wallis

3. **Нормальны ли данные?**
   - Да → параметрические тесты (t-test, Пирсон)
   - Нет → непараметрические (Mann-Whitney, Спирман)
   - Не уверены → используйте непараметрические или bootstrap

### ⚠️ Важные замечания:

- **p < 0.05** не означает "результат важен", только "вряд ли случайность"
- **Размер эффекта** важнее статистической значимости
- **Множественное тестирование** требует поправок (Bonferroni, FDR)
- **Корреляция ≠ Причинность** (всегда помните!)

---
## 🎓 Подведение итогов

**Сегодня мы изучили:**

✅ **T-test** — сравнение средних двух групп  
✅ **Mann-Whitney** — непараметрическая альтернатива t-теста  
✅ **Корреляцию** — оценка связи между переменными  
✅ **Bootstrap** — универсальный метод оценки неопределённости  

**Ключевые концепции:**
- Нулевая и альтернативная гипотезы
- P-value и уровень значимости
- Доверительные интервалы
- Параметрические vs непараметрические тесты

**Следующие шаги:**
1. Примените эти методы к своим данным
2. Изучите A/B тестирование
3. Узнайте про множественное тестирование
4. Погрузитесь в причинный вывод (causal inference)

### 📖 Полезные ресурсы:
- [SciPy Stats Documentation](https://docs.scipy.org/doc/scipy/reference/stats.html)
- [Seeing Theory (визуализация статистики)](https://seeing-theory.brown.edu/)
- [StatQuest (YouTube канал)](https://www.youtube.com/c/joshstarmer)

In [ ]:
print("🎉 Поздравляем! Вы завершили занятие по статистическим тестам!")
print("💪 Теперь вы можете проверять гипотезы на данных как настоящий аналитик!")